In [1]:
import os
import sys
import math
import glob
import tqdm
import random
import numpy as np
from tqdm import tqdm
from time import sleep
from shutil import copy

In [2]:
import pandas as pd
import xml.etree.cElementTree as ET

In [3]:
import cv2
import dlib
from imutils import face_utils
from skimage.feature import hog
from skimage import data,exposure

In [4]:
path_folder_bagamoyo_data = '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/'

In [5]:
path_folder_subject_videos = path_folder_bagamoyo_data + 'bagamoyo_videos/subject_sorted_videos/'

In [6]:
path_folder_framewise_videos = path_folder_bagamoyo_data + 'bagamoyo_frames_folder_wise/'

In [7]:
list_path_subject_folders = glob.glob(path_folder_subject_videos+'/*/')
list_path_subject_folders

['/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/1/',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/10/',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/11/',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/12/',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/13/',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/2/',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/3/',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/4/',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/5/',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/6/',
 '/media/amogh/Stuff/CMU/datasets/bagamoyo_data/bagamoyo_videos/subject_sorted_videos/

In [8]:
path_folder_data = 'faces/'

Subjects to use for training and testing

In [9]:
list_training_subjects = []
list_testing_subjects = []

### Save data (frames)

In [10]:
def getListSubjectVideos(path_folder_subject_videos, subject_num):
    """
    Returns the path to a particular subject's videos given the path to folder of subject videos and subject number
    """
    path_subject_videos = path_folder_subject_videos + str(subject_num) +'/'
    list_subject_videos = glob.glob(path_subject_videos + '/*')
    return list_subject_videos

In [11]:
def getPathFolderFrames(base_name_video, path_folder_frames_wise):
    """
    Returns the path of the folder with frames given the name of the video.
    Eg- VIDEO_00-male-surprise_20180517_050935_780615161 is the videlo basename
    returns the path <path_to_bagamoyo_frames_folder_wise>/VIDEO_00-male-surprise_20180517_050935_780615161 (5-29-2018 10-05-16 AM)
    """
    list_folders = glob.glob(path_folder_frames_wise + '/*')
    path_frames_folder = ''
    for folder in list_folders:
        if base_name_video in folder:
            path_frames_folder = folder
    return path_frames_folder    

In [12]:
def generateTrainTestData(list_train_test_subjects,path_folder_subject_videos, path_folder_data):
    """
    Copies the images from the train and test folders in the faces folder given the list with number of subjects.
    """
    for subject_num in list_train_test_subjects:
        list_subject_videos = getListSubjectVideos(path_folder_subject_videos,subject_num)
        #get images from the relevant folder and save in faces
        for video in tqdm(list_subject_videos):
            basename_folder_video = os.path.splitext(os.path.basename(video))[0]
            #get the folder with the frames
            path_folder_frames = getPathFolderFrames(basename_folder_video,path_folder_framewise_videos)
            list_path_frames = glob.glob(path_folder_frames + '/*.jpg')
            for path_frame in list_path_frames:
                print(path_frame, path_folder_data)
                copy(path_frame, path_folder_data)

### Generate the training.xml and testing.xml

Aim- 
training subjects
 - get video names of these subjects
    - get frame names under these subjects
    - get boxes under these frames (from data xml(generate for all videos in a folder))
        - put in training boxes.
        - put in test boxes.        

Generate the xml for complete data from csv using the script written earlier.
Given the subject numbers, take those videos, get all frames in them, get all elements in them and make training and test xml before starting the script.

In [24]:
random.sample(list(range(10)),2)

[2, 9]

In [46]:
def generateDataXML(list_subject_num,path_data_xml,path_output,subsample=True):
    #parsing data and creating dictionary
    tree = ET.parse(path_data_xml)
    root = tree.getroot()
    dic_file_to_image = {}
    for image in root.iter('image'):
        dic_file_to_image[image.attrib['file']] = image
#     print((dic_file_to_image.keys()))
    
    #Writing an XML
    root = ET.Element("dataset")
    name = ET.SubElement(root, "name").text = "Labelled faces"
    comment = ET.SubElement(root, "comment").text = "These are labelled images from Bagamoyo"
    images = ET.SubElement(root, "images")
    
    #get the list of images
    for subject_num in list_subject_num:
        list_subject_videos = getListSubjectVideos(path_folder_subject_videos,subject_num)
        #get images from the relevant folder and save in faces
        for video in tqdm(list_subject_videos):
            basename_folder_video = os.path.splitext(os.path.basename(video))[0]
            #get the folder with the frames
            path_folder_frames = getPathFolderFrames(basename_folder_video,path_folder_framewise_videos)
            list_path_frames = glob.glob(path_folder_frames + '/*.jpg')
            print("{} frames present".format(len(list_path_frames)))
            i=0
            for path_frame in list_path_frames:
#                 get the frame tag in xml, if present append.
                image_file_name = os.path.basename(path_frame)
                if image_file_name in dic_file_to_image:
                    im_node = dic_file_to_image[image_file_name]
                    images.append(im_node)
                    pass
                else:
                    i+=1
            print("{} frames with no detection".format(i))
    num_images = len(images)
    print(num_images)
    index_ignore = random.sample(list(range(num_images)), num_images-2000)
    print(len(index_ignore))
    for j,image in enumerate(images):
        if j in index_ignore:
            par = image.getparent()
            par.remove(image)
    print(len(images))
    #get 2000 random samples.
    #write the XML file
#     tree = ET.ElementTree(root)
#     dest_name = path_output
#     tree.write(dest_name)

In [47]:
generateDataXML([1,2,3,4,5,6],'/media/amogh/Stuff/CMU/datasets/bagamoyo_data/openface_outputs/library/openface_199_folders_75_confidence.xml','training.xml')

100%|██████████| 13/13 [00:00<00:00, 205.83it/s]

186 frames present
19 frames with no detection
119 frames present
0 frames with no detection
136 frames present
14 frames with no detection
142 frames present
8 frames with no detection
257 frames present
136 frames with no detection
150 frames present
4 frames with no detection
135 frames present
0 frames with no detection
100 frames present
99 frames with no detection
420 frames present
315 frames with no detection
134 frames present
4 frames with no detection
143 frames present
2 frames with no detection
140 frames present
3 frames with no detection
172 frames present
7 frames with no detection
233 frames present
11 frames with no detection
149 frames present
19 frames with no detection
131 frames present
1 frames with no detection
161 frames present
2 frames with no detection
129 frames present
0 frames with no detection
248 frames present
131 frames with no detection
105 frames present
3 frames with no detection
141 frames present
1 frames with no detection
155 frames present
1 fr


100%|██████████| 14/14 [00:00<00:00, 217.50it/s]

124 frames present
0 frames with no detection
149 frames present
11 frames with no detection
145 frames present
0 frames with no detection
608 frames present
467 frames with no detection
153 frames present
0 frames with no detection
46 frames present
46 frames with no detection
132 frames present
7 frames with no detection
165 frames present
0 frames with no detection
148 frames present
2 frames with no detection
128 frames present
0 frames with no detection
135 frames present
0 frames with no detection
116 frames present
0 frames with no detection
137 frames present
4 frames with no detection
128 frames present
0 frames with no detection
106 frames present
0 frames with no detection
197 frames present
11 frames with no detection
130 frames present
1 frames with no detection
152 frames present
4 frames with no detection
40 frames present
40 frames with no detection
227 frames present
6 frames with no detection
388 frames present
10 frames with no detection
212 frames present
6 frames w

AttributeError: getparent

In [15]:
generateDataXML([7,8,9],'/media/amogh/Stuff/CMU/datasets/bagamoyo_data/openface_outputs/library/openface_199_folders_75_confidence.xml','testing.xml')

 36%|███▋      | 4/11 [00:00<00:00, 14.16it/s]

150 frames present
4 frames with no detection
0 frames present
0 frames with no detection
115 frames present
15 frames with no detection
143 frames present
10 frames with no detection


 64%|██████▎   | 7/11 [00:00<00:00, 12.06it/s]

320 frames present
163 frames with no detection
118 frames present
4 frames with no detection
105 frames present
7 frames with no detection


 91%|█████████ | 10/11 [00:00<00:00, 12.10it/s]

...

111 frames present
0 frames with no detection
102 frames present
99 frames with no detection
24 frames present
24 frames with no detection
176 frames present
4 frames with no detection
117 frames present
0 frames with no detection


 80%|████████  | 8/10 [00:00<00:00, 17.80it/s]

87 frames present
87 frames with no detection
93 frames present
93 frames with no detection


100%|██████████| 10/10 [00:00<00:00, 12.94it/s]

106 frames present
0 frames with no detection
130 frames present
3 frames with no detection


In [18]:
def try1():
    tree = ET.parse('/home/amogh/cmu/dlib/examples/faces/training.xml')
    root = tree.getroot()
    dic = {}
    for image in root.iter('image'):
        dic[image.attrib['file']] = image
    print(dic)
try1()

{'2008_001322.jpg': <Element 'image' at 0x7ff7b77e4ab0>, '2007_007763.jpg': <Element 'image' at 0x7ff7b77e4e10>, '2008_002079.jpg': <Element 'image' at 0x7ff7b77e4030>, '2008_001009.jpg': <Element 'image' at 0x7ff7b77e4a20>}


### Training